## Model using permutation as representation

In [2]:
from sympy.combinatorics.permutations import Permutation

#____________________________________Model_________________________________________________

def evenPermRandom(n):
    """generates a random permutation of length n using a black-box approach"""
    cand = Permutation.random(n)
    while cand.signature() == -1: 
        cand = Permutation.random(n)
    return cand

class Model:  
    def __init__(self, width, height):
        self.n = width*height
        self.width = width
        self.blank_number = self.n-1    
        #the position of the blank square
        self.blank_pos = self.n-1
        
        #the .observer attribute needs to be initialized afterwards,
        #before any operation calling update is executed
        #added here in unitialized state for clarity
        self.observer = None
        
        #identity needed to check if you've won
        self.I = Permutation(range(self.n))
        #start with identity
        self.perm = Permutation(range(self.n))        

    def shuffle(self):
        """reset the permutation to a random (solvable) one"""
        self.perm = Permutation(self.n-1) * evenPermRandom(self.n-1)
        self.blank_pos=self.n-1
        self.observer.update(range(self.n))#notify change 

    
    def adjacent(self, i,j): 
        dist = abs(i - j) 
        #same row or same row or same column
        #for same row must check _actually_ same row
        return (dist == self.width or dist == 1 and i // self.width == j // self.width)

    def processMove(self,i):
        if self.adjacent(i, self.blank_pos) :
            self.perm = Permutation(i,self.blank_pos)*self.perm
            self.observer.update([i,self.blank_pos]) #notify change
            self.blank_pos = i
            if self.perm == self.I:
                print ("Congrats!") #belongs in View 



# n×m Sliding Puzzle Game

In [17]:
from ipywidgets import Button, GridBox, HBox, Layout, ButtonStyle, BoundedIntText, VBox


#_________________________________________View_______________________________________________________
normal_b_style=ButtonStyle()
blank_b_style=ButtonStyle()
blank_b_style.button_color="lightgreen"

button_layout=Layout(
                      height='50px'
                    , width='50px'
)

#generate the formatting text for the GridBox
def auto_string(l):
    autos = ['auto' for i in range(l)]
    return ' '.join(autos)

class MyButton(Button):
    def __init__(self,pos,**kwargs):
        super().__init__(**kwargs)
        self.pos = pos

class GameView:        
  
    def __init__(self,model):
        self.model = model
        self.buttons = [MyButton(layout=button_layout
                            , pos=i
                            ) 
                       for i in range(self.model.n)
                          ] 

        for b in self.buttons:
            b.on_click(lambda b: self.model.processMove(b.pos)) #controller
            
        self.game = GridBox(
            children=self.buttons,
            layout=Layout(
              grid_template_columns=auto_string(self.model.width)
            , justify_content='flex-start'
             )
           )
        self.update(range(self.model.n))
        self.shuffleB = Button(description = "shuffle")
        self.shuffleB.on_click(lambda b: self.model.shuffle()) #controller


    def update(self,locations): #called by model
        """mutate the view at `locations` to reflect the model's state"""
        for l in locations:
            lp = l^self.model.perm
            if lp == self.model.blank_number:
                self.buttons[l].style = blank_b_style
                self.buttons[l].description = " "
            else:
                self.buttons[l].style = normal_b_style
                self.buttons[l].description = str(lp)
          
    
#Dimension input & generation menu
width_input = BoundedIntText(description="width"
                            , min=1
                            , max=10
                           )
height_input = BoundedIntText(description="height"
                              , min=1
                              , max=10
                   )
generateB = Button(description = "Generate")

genMenu = HBox([width_input, height_input, generateB])

allView = VBox([genMenu])

def generate():
    width = width_input.value
    height = height_input.value
    
    model = Model(width,height)
    view = GameView(model)
    
    model.observer = view
    allView.children = (allView.children[0], HBox([view.game,view.shuffleB]))
    
    
generateB.on_click(lambda b: generate())

Permutation.print_cyclic = False
allView

Congrats!
